# 数据聚合与分组运算

## GroupBy机制

分组——应用——合并

###  分组

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.DataFrame({'key1':['a','a','b','b','a'],
                'key2':['one','two','one','two','one'],
                'data1':np.random.randn(5),
                'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.679618,0.789840
1,a,two,0.531742,0.886568
2,b,one,-1.706796,-0.091565
3,b,two,-1.176277,-1.287026
4,a,one,-0.510038,-2.577218


In [3]:
# 按照key1进行分组，并计算data1的平均值
# 访问data1，并根据key1调用groupby

In [4]:
grouped=df['data1'].groupby(df['key1'])
grouped
#grouped 是一个GroupBy 对象，实际上没有进行任何计算

In [5]:
grouped.mean()

key1
a    0.233774
b   -1.441536
Name: data1, dtype: float64

In [6]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one     0.084790
      two     0.531742
b     one    -1.706796
      two    -1.176277
Name: data1, dtype: float64

In [7]:
means.unstack()
# 把多元索引变成DF

key2,one,two
key1,,
a,0.084790,0.531742
b,-1.706796,-1.176277


In [8]:
states=np.array(['ohio','califonia','califonia',
                  'ohio','ohio'])
years=np.array([2005,2005,2006,2005,2006])

df['data1'].groupby([states,years]).mean()

califonia  2005    0.531742
           2006   -1.706796
ohio       2005   -0.248330
           2006   -0.510038
Name: data1, dtype: float64

In [9]:
df

,key1,key2,data1,data2
0,a,one,0.679618,0.789840
1,a,two,0.531742,0.886568
2,b,one,-1.706796,-0.091565
3,b,two,-1.176277,-1.287026
4,a,one,-0.510038,-2.577218


In [10]:
df.groupby('key1').mean()
# groupby()后加分组名称

,data1,data2
key1,,
a,0.233774,-0.300270
b,-1.441536,-0.689295


In [11]:
df.groupby(['key1','key2']).mean()

# 看成坐标定位

data1     data2
key1 key2                    
a    one   0.084790 -0.893689
     two   0.531742  0.886568
b    one  -1.706796 -0.091565
     two  -1.176277 -1.287026

In [12]:
# key2 非数值 麻烦列

In [13]:
df.groupby(['key1','key2']).size()

# 返回一个含有分组大小的S  缺失值会被剔除

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 对分组进行迭代

In [14]:
# 为了获取分组内容

In [15]:
df

,key1,key2,data1,data2
0,a,one,0.679618,0.789840
1,a,two,0.531742,0.886568
2,b,one,-1.706796,-0.091565
3,b,two,-1.176277,-1.287026
4,a,one,-0.510038,-2.577218


In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    

a
  key1 key2     data1     data2
0    a  one  0.679618  0.789840
1    a  two  0.531742  0.886568
4    a  one -0.510038 -2.577218
b
  key1 key2     data1     data2
2    b  one -1.706796 -0.091565
3    b  two -1.176277 -1.287026


In [17]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.679618  0.789840
4    a  one -0.510038 -2.577218
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.531742  0.886568
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.706796 -0.091565
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.176277 -1.287026


In [18]:
pieces=dict(list(df.groupby('key1')))
pieces

# 用字典形式把groupby ('key1') 内容表现出来

{'a':   key1 key2     data1     data2
 0    a  one  0.679618  0.789840
 1    a  two  0.531742  0.886568
 4    a  one -0.510038 -2.577218, 'b':   key1 key2     data1     data2
 2    b  one -1.706796 -0.091565
 3    b  two -1.176277 -1.287026}

In [19]:
df.groupby('key1')
# data1 data2 用key1 排列


In [20]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one  0.679618  0.789840
  1    a  two  0.531742  0.886568
  4    a  one -0.510038 -2.577218), ('b',   key1 key2     data1     data2
  2    b  one -1.706796 -0.091565
  3    b  two -1.176277 -1.287026)]

In [21]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.706796,-0.091565
3,b,two,-1.176277,-1.287026


 groupby默认在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组

In [22]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [23]:
grouped=df.groupby(df.dtypes,axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.679618  0.789840
1  0.531742  0.886568
2 -1.706796 -0.091565
3 -1.176277 -1.287026
4 -0.510038 -2.577218
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 选取一列或列的子集

### 表达方式一

In [24]:
df.groupby('key1')['data1']
df.groupby('key1')['data2']

### 大数据 （语法糖）

In [25]:
s_grouped=df['data1'].groupby(df['key1']).mean()
# df['data2'].groupby(df['key1'])
s_grouped

key1
a    0.233774
b   -1.441536
Name: data1, dtype: float64

有两种方法来选取一列或列的子集，其一是直接用索引，其二是用两组索引来表示

## 通过字典或Series进行分组

In [26]:
df

,key1,key2,data1,data2
0,a,one,0.679618,0.789840
1,a,two,0.531742,0.886568
2,b,one,-1.706796,-0.091565
3,b,two,-1.176277,-1.287026
4,a,one,-0.510038,-2.577218


In [27]:
df.iloc[1:2,[3]]=np.nan

In [28]:
df

,key1,key2,data1,data2
0,a,one,0.679618,0.789840
1,a,two,0.531742,NaN
2,b,one,-1.706796,-0.091565
3,b,two,-1.176277,-1.287026
4,a,one,-0.510038,-2.577218


In [29]:
mapping={'data1':'red','data2':'green'}

In [30]:
by_column=df.groupby(mapping,axis=1)
by_column.sum()

,green,red
0,0.789840,0.679618
1,0.000000,0.531742
2,-0.091565,-1.706796
3,-1.287026,-1.176277
4,-2.577218,-0.510038


In [31]:
map_series=pd.Series(mapping)
map_series

data1      red
data2    green
dtype: object

In [32]:
df.groupby(map_series,axis=1).count()

,green,red
0,1,1
1,0,1
2,1,1
3,1,1
4,1,1


In [33]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.groupby(mapping, axis=1)   # 通过mapping 来进行groupby
# by_column.sum()
people.iloc[2:3, [1, 2]] = np.nan 

groupby(self, by, axis, level, as_index, 
        sort, group_keys, squeeze, observed, **kwargs)

In [34]:
people

,a,b,c,d,e
Joe,0.450761,0.764692,-0.306041,-0.598232,0.776740
Steve,1.468091,-2.168096,-1.407804,0.300345,1.171628
Wes,-1.315382,NaN,NaN,0.368946,2.254980
Jim,-0.020549,-2.515832,-0.086434,0.025917,0.007369
Travis,1.954466,0.331475,-1.874397,2.433301,-1.122247


In [35]:
by_column.sum()

,blue,red
Joe,-0.904272,1.992192
Steve,-1.107460,0.471622
Wes,0.368946,0.939597
Jim,-0.060517,-2.529012
Travis,0.558904,1.163694


In [36]:
people.loc['Wes','a']+people.loc['Wes','e']

0.9395974271438121

##  对函数进行分组

In [37]:
people.groupby(len).sum()

# 通过英文名长度来划分
# len 在index上被调用了 

,a,b,c,d,e
3,-0.885170,-1.751141,-0.392475,-0.203368,3.039088
5,1.468091,-2.168096,-1.407804,0.300345,1.171628
6,1.954466,0.331475,-1.874397,2.433301,-1.122247


In [38]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [39]:
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -1.315382  0.764692 -0.306041 -0.598232  0.776740
  two -0.020549 -2.515832 -0.086434  0.025917  0.007369
5 one  1.468091 -2.168096 -1.407804  0.300345  1.171628
6 two  1.954466  0.331475 -1.874397  2.433301 -1.122247

In [40]:
people

,a,b,c,d,e
Joe,0.450761,0.764692,-0.306041,-0.598232,0.776740
Steve,1.468091,-2.168096,-1.407804,0.300345,1.171628
Wes,-1.315382,NaN,NaN,0.368946,2.254980
Jim,-0.020549,-2.515832,-0.086434,0.025917,0.007369
Travis,1.954466,0.331475,-1.874397,2.433301,-1.122247


In [41]:
# 分析过程：

# 3 joe  wes jim  1*2  2*1
# 5 steve  1
# 6 travis  2



## 根据索引级别分组

使用level关键字传递级别序号或名字

In [42]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], 
                                     [1, 3, 5, 1, 3]], 
                                    names=['city', 'tenor'])
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           codes=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['city', 'tenor'])

In [43]:
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.870572  0.838323  0.752899 -0.899499  0.671461
1     -1.068325 -1.438251  1.370203 -0.516900  2.175559
2     -0.359515 -0.239287  0.141495 -0.497289  1.047806
3      0.434579  0.103425  0.201163  1.395034  0.478928

In [44]:
hier_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 数据聚合

In [45]:
df

,key1,key2,data1,data2
0,a,one,0.679618,0.789840
1,a,two,0.531742,NaN
2,b,one,-1.706796,-0.091565
3,b,two,-1.176277,-1.287026
4,a,one,-0.510038,-2.577218


In [46]:
grouped=df.groupby('key1')

grouped['data1'].quantile(0.9)

key1
a    0.650042
b   -1.229329
Name: data1, dtype: float64

In [47]:
def peak_to_peak(arr):
    return arr.max()-arr.min()
grouped.agg(peak_to_peak)

#  agg()聚合

,data1,data2
key1,,
a,1.189655,3.367057
b,0.530519,1.195461


In [48]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.233774  0.648389 -0.510038  0.010852  0.531742  0.605680   
b      2.0 -1.441536  0.375134 -1.706796 -1.574166 -1.441536 -1.308907   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.679618   2.0 -0.893689  2.380869 -2.577218 -1.735453 -0.893689   
b    -1.176277   2.0 -0.689295  0.845319 -1.287026 -0.988161 -0.689295   

                          
           75%       max  
key1                      
a    -0.051925  0.789840  
b    -0.390430 -0.091565

## 面向列的多函数应用

In [49]:
tips=pd.read_csv('G:\work\pydata-book-2nd-edition\examples\\tips.csv')

In [50]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [51]:
tips['tip_pct']=tips['tip']/tips['total_bill']
# 计算

In [52]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [53]:
grouped=tips.groupby(['day','smoker'])
# day  smoker

In [54]:
grouped_pct=grouped['tip_pct']

# 取出序列

In [55]:
for name, group in grouped_pct:
    print(name)
    print(group,'\n')

('Fri', 'No')
91     0.155625
94     0.142857
99     0.120385
223    0.187735
Name: tip_pct, dtype: float64 

('Fri', 'Yes')
90     0.103555
92     0.173913
93     0.263480
95     0.117750
96     0.146628
97     0.124688
98     0.142789
100    0.220264
101    0.195059
220    0.180921
221    0.259314
222    0.223776
224    0.117735
225    0.153657
226    0.198216
Name: tip_pct, dtype: float64 

('Sat', 'No')
19     0.162228
20     0.227679
21     0.135535
22     0.141408
23     0.192288
24     0.160444
25     0.131387
26     0.149589
27     0.157604
28     0.198157
29     0.152672
30     0.151832
31     0.136240
32     0.199203
33     0.118415
34     0.183915
35     0.149626
36     0.122624
37     0.181335
38     0.123596
39     0.159898
40     0.139651
57     0.056797
59     0.139424
64     0.150085
65     0.156873
66     0.150152
68     0.099357
70     0.163894
71     0.175747
74     0.149355
75     0.118934
104    0.195029
108    0.206140
110    0.214286
111    0.137931
212    0.1862

In [56]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [57]:
grouped_pct.agg(['mean','std',peak_to_peak,'max','min'])

mean       std  peak_to_peak       max       min
day  smoker                                                      
Fri  No      0.151650  0.028123      0.067349  0.187735  0.120385
     Yes     0.174783  0.051293      0.159925  0.263480  0.103555
Sat  No      0.158048  0.039767      0.235193  0.291990  0.056797
     Yes     0.147906  0.061375      0.290095  0.325733  0.035638
Sun  No      0.160113  0.042347      0.193226  0.252672  0.059447
     Yes     0.187250  0.154134      0.644685  0.710345  0.065660
Thur No      0.160298  0.038774      0.193350  0.266312  0.072961
     Yes     0.163863  0.039389      0.151240  0.241255  0.090014

In [58]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

# (name, fuction)的元组 组成的列表，可以为其命名

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

一个含有多个函数的list应用到所有列上

In [59]:
fuctions=['count','mean','max']

In [60]:
result=grouped['tip_pct','total_bill'].agg(fuctions)

In [61]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [62]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

### 总结前面两种用法

In [63]:
ftuples=[('Durchschnitt','mean'),('abweichung',np.var)]
#元组形式表示

In [64]:
grouped['tip_pct','total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt abweichung Durchschnitt  abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [65]:
grouped.agg({'tip':np.max,'size':'sum'})
# 字典形式表现

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [66]:
grouped['tip_pct','total_bill'].agg({'tip':np.max,'size':'sum'})

# 用字典形式和元组形式存在区别 
# 元组形式置于第二层索引，而字典形式置于第一层

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


tip                 size           
              tip_pct total_bill   tip_pct total_bill
day  smoker                                          
Fri  No      0.187735      22.75  0.606602      73.68
     Yes     0.263480      40.17  2.621746     252.20
Sat  No      0.291990      48.33  7.112145     884.78
     Yes     0.325733      50.81  6.212055     893.62
Sun  No      0.252672      48.17  9.126438    1168.88
     Yes     0.710345      45.35  3.557756     458.28
Thur No      0.266312      41.19  7.213414     770.09
     Yes     0.241255      43.11  2.785676     326.24

In [67]:
grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## 以没有行索引的形式 返回聚合数据

In [68]:
tips.groupby(['day','smoker'], as_index=False).mean()

# 使用as_index=False 返回一个没有行索引的df

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [69]:
means=tips.groupby(['day','smoker']).mean()

In [70]:
means.reset_index()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 apply：一般性的“拆分—应用—合并”

groupby

agg()

In [78]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [79]:
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [81]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [85]:
tips.groupby(['smoker','day']).apply(top,n=1,
                                     column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [87]:
result=tips.groupby('smoker')['tip_pct'].describe()

In [89]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [90]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [93]:
# f=lambda x: x.describe()
# grouped.apply(y)

### 禁止分组键

In [96]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析

In [104]:
frame=pd.DataFrame({'data1':np.random.randn(1000),
                   'data2':np.random.randn(1000)})
# frame

In [105]:
quartiles=pd.cut(frame.data1, 4)

# 将数据拆分成多块（cut  qcut）切分成四块

In [106]:
quartiles[:10]

0    (-1.789, -0.0851]
1     (-0.0851, 1.619]
2    (-1.789, -0.0851]
3    (-1.789, -0.0851]
4    (-1.789, -0.0851]
5       (1.619, 3.322]
6     (-0.0851, 1.619]
7    (-1.789, -0.0851]
8    (-1.789, -0.0851]
9     (-0.0851, 1.619]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.5, -1.789] < (-1.789, -0.0851] < (-0.0851, 1.619] < (1.619, 3.322]]

In [108]:
def get_stats(group):
    return{'min':group.min(),'max':group.max(),
           'count':group.count(),'mean':group.mean()}

grouped=frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.5, -1.789]",52.0,1.519929,-0.053097,-1.896511
"(-1.789, -0.0851]",447.0,3.078638,-0.058695,-2.992176
"(-0.0851, 1.619]",434.0,2.945567,0.099392,-2.730231
"(1.619, 3.322]",67.0,2.122536,-0.057016,-2.549949


In [111]:
grouping=pd.qcut(frame.data1,10,labels=False) # 分成十份

grouped=frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.444500,-0.028756,-2.283067
1,100.0,2.305323,0.043473,-2.273482
2,100.0,2.557900,-0.059632,-2.543463
3,100.0,3.078638,-0.086747,-2.418926
4,100.0,2.188233,-0.154907,-2.992176
5,100.0,2.945567,0.152721,-1.535309
6,100.0,2.097128,0.040867,-2.730231
7,100.0,2.664025,-0.024187,-2.467767
8,100.0,2.171265,0.256088,-2.218095


### 示例 用特定分组的值填充缺失值

In [117]:
s=pd.Series(np.random.randn(6))
s[::2]=np.nan
s

0         NaN
1   -0.309056
2         NaN
3    0.816064
4         NaN
5   -0.365656
dtype: float64

In [119]:
s.fillna(s.mean())

0    0.047117
1   -0.309056
2    0.047117
3    0.816064
4    0.047117
5   -0.365656
dtype: float64

In [121]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [123]:
group_key=['east']*4+['west']*4
# 产生了两个列表

In [125]:
data=pd.Series(np.random.randn(8), index=states)

In [127]:
data

Ohio         -1.330776
New York      0.835234
Vermont      -0.911564
Florida       1.426156
Oregon        0.129670
Nevada        0.398032
California   -0.930361
Idaho         0.376273
dtype: float64

In [129]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.330776
New York      0.835234
Vermont            NaN
Florida       1.426156
Oregon        0.129670
Nevada             NaN
California   -0.930361
Idaho              NaN
dtype: float64

In [132]:
data.groupby(group_key).mean()

east    0.310205
west   -0.400345
dtype: float64

In [139]:
import numpy as np
fill_mean=lambda g :g.fillna(g.mean())

data.groupby(group_key).apply(fill_mean)

Ohio         -1.330776
New York      0.835234
Vermont       0.310205
Florida       1.426156
Oregon        0.129670
Nevada       -0.400345
California   -0.930361
Idaho        -0.400345
dtype: float64

In [143]:
fill_values={'east':0.5,'west':-1}
fill_func=lambda g :g.fillna(fill_values[g.name])
# 分组具有name属性

data.groupby(group_key).apply(fill_func)

Ohio         -1.330776
New York      0.835234
Vermont       0.500000
Florida       1.426156
Oregon        0.129670
Nevada       -1.000000
California   -0.930361
Idaho        -1.000000
dtype: float64

### 示例：随机采样和排序

In [145]:
suits = ['H', 'S', 'C', 'D']
# 红桃 黑桃 梅花 方块

card_val = (list(range(1, 11)) + [10] * 3) * 4
# 长度13*4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [148]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [154]:
def draw(deck,n=5):
    return deck.sample(n)

In [155]:
draw(deck)

QH    10
QS    10
6S     6
8S     8
2C     2
dtype: int64

In [159]:
get_suit= lambda card: card[-1]

deck.groupby(get_suit).apply(draw,n=2)

C  9C      9
   QC     10
D  6D      6
   2D      2
H  3H      3
   10H    10
S  7S      7
   4S      4
dtype: int64

In [161]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

2C      2
6C      6
10D    10
JD     10
2H      2
KH     10
JS     10
7S      7
dtype: int64

### 分组加权和相关系数

In [163]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-1.939576,0.362622
1,a,0.743609,0.006888
2,a,-0.624908,0.238407
3,a,-0.585148,0.377442
4,b,-0.116930,0.955641
5,b,0.800684,0.393081
6,b,-1.930018,0.993888
7,b,0.562870,0.875096


In [166]:
grouped=df.groupby('category')

get_wavg= lambda g :np.average(g['data'],weights=g['weights'])

grouped.apply(get_wavg)


category
a   -1.083922
b   -0.379980
dtype: float64

In [172]:
close_px=pd.read_csv('G:\work\pydata-book-2nd-edition\examples\\stock_px_2.csv',
                    parse_dates=True, index_col=0)

In [174]:
close_px.info

<bound method DataFrame.info of               AAPL   MSFT    XOM      SPX
2003-01-02    7.40  21.11  29.22   909.03
2003-01-03    7.45  21.14  29.24   908.59
2003-01-06    7.45  21.52  29.96   929.01
2003-01-07    7.43  21.93  28.95   922.93
2003-01-08    7.28  21.31  28.83   909.93
2003-01-09    7.34  21.93  29.44   927.57
2003-01-10    7.36  21.97  29.03   927.57
2003-01-13    7.32  22.16  28.91   926.26
2003-01-14    7.30  22.39  29.17   931.66
2003-01-15    7.22  22.11  28.77   918.22
2003-01-16    7.31  21.75  28.90   914.60
2003-01-17    7.05  20.22  28.60   901.78
2003-01-21    7.01  20.17  27.94   887.62
2003-01-22    6.94  20.04  27.58   878.36
2003-01-23    7.09  20.54  27.52   887.34
2003-01-24    6.90  19.59  26.93   861.40
2003-01-27    7.07  19.32  26.21   847.48
2003-01-28    7.29  19.18  26.90   858.54
2003-01-29    7.47  19.61  27.88   864.36
2003-01-30    7.16  18.95  27.37   844.61
2003-01-31    7.18  18.65  28.13   855.70
2003-02-03    7.33  19.08  28.52   860.32
20

In [176]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


### 组级别的线性回归

In [72]:
import statsmodels.api as sm

In [73]:
def regress(data,yvar,xvars):
    Y=data[yvar]
    x=data[xvars]
    x['intercpt']=1# 新建一个常数项，令其为1
    result=sm.OLS(Y,X).fit()
    return result.params

In [74]:
by_year.apply(regress,'AAPL',['SPX'])

NameError: name 'by_year' is not defined

# 透视表和交叉表

pivot table

In [ ]:
tips.pivot_table(index=['day','smoker'])

# pivot_table 默认聚合为分组平均数

In [ ]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                 columns='smoker',margins=True)

In [ ]:
#  margins=True 添加小计

In [ ]:
tips.pivot_table('tip_pct', index=['time','smoker'])